In [1]:
from bs4 import BeautifulSoup
import urllib.request
import ssl
import time
import json

database = "database.json"

In [2]:
def get_datetime_page(soup):
    try:
        return soup.find_all("table")[0].find_all("div")[0].text
    except:
        return ""
    
def get_table_data(soup):
    """
    @param soup object of a page
    @return list of dict : [{...}, {...}, {...}]
    each object in list represent each row in the table
    
    {
        "airline": "",
        "flight": "",
        "from": "",
        "scheduled": "",
        "actual": "",
        "gate": "",
    
    }
    """
    table = soup.find_all("table", class_="my_flight")
    parsed_table = list()
    for row in table:
        if not row.contents: # malform row
            continue
        parsed_row = dict()
        columns = row.contents[0].find_all("td")
        if len(columns) != 10: # malform row
            continue
        parsed_row["airline"] = columns[1].text
        parsed_row["flight"] = columns[2].text
        parsed_row["airport"] = columns[4].text
        parsed_row["city"] = columns[5].text
        parsed_row["scheduled"] = columns[6].text
        parsed_row["actual"] = columns[7].text
        parsed_row["gate"] = columns[8].text
        parsed_row["status"] = columns[9].text
        parsed_table.append(parsed_row)
            
    return parsed_table


def scraped():
    """
    @return {"<date as shown on the top of the page>" : <list of dict as return from get_table_data>}
    """
    ssl._create_default_https_context = ssl._create_unverified_context
    url = "https://www.airport-la.com/lax/arrivals?t="
    htmls = dict()

    for i in range(-10,21):
        with urllib.request.urlopen(url + str(i)) as response:
            html = response.read()
        soup = BeautifulSoup(html)
        date = get_datetime_page(soup)
        if date:
            data = get_table_data(soup)
            htmls[date] = data     
    return htmls 


In [3]:
# Run Daily

now = time.time()

with open(database, "r") as f:
    old = json.load(f)
    
old[now] = scraped()


with open(database, "w") as f:
    json.dump(old, f)